Here we will do the four kpi's (Key performance indicator) for both control and test group and we will compare them so that we know how the new design performed.

#### Let's do the four kpi's for the Test_group "Completion rate , Average time spent, Time spent on each step, Error rate" for Test group

In [2]:
# Load necessary libraries
import pandas as pd
import numpy as np
from collections import defaultdict
from datetime import datetime
import time

In [8]:
# load the data
test_sorted = pd.read_csv("test_sorted.csv")
test_sorted.head(2)

,client_id,visitor_id,visit_id,process_step,date_time,date,time,year,month,day,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,age_category,variation_test
0,555,402506806_56087378777,637149525_38041617439_716659,confirm,4/15/2017 13:00,"2017, 00, 15","13, 00, 34",2017,4,15,3,46,30,U,2,25454,2,6,14-30,Test
1,555,402506806_56087378777,637149525_38041617439_716659,start,4/15/2017 12:57,"2017, 57, 15","12, 57, 56",2017,4,15,3,46,30,U,2,25454,2,6,14-30,Test


#### Completion rate

In [9]:
# Calculate for the total confirmed process step
process_step_confirm = len(test_sorted[test_sorted['process_step'] == 'confirm'])
process_step_confirm

25594

In [10]:
# Calculate for the total start process step
process_step_start = len(test_sorted[test_sorted['process_step'] == 'start'])
process_step_start

55753

In [11]:
# Calculate for the completion rate
completion_rate = round((process_step_confirm/process_step_start) * 100)
completion_rate

46

#### Average time spent

In [15]:
# Open dictionary to store time differences for each client ID
test_sorted2 = test_sorted[['client_id','time']]
test_sorted2_list = test_sorted2.values.tolist()

time_diffs = defaultdict(list)

# Parse timestamps and group by client ID
for client_id, time in test_sorted2_list:
    time_diffs[client_id].append(datetime.strptime(time, '%H, %M, %S'))

In [16]:
# Calculate time differences for each client ID
time_diffrence1 = []
time_diffrence2 = []
for client_id, time in time_diffs.items():
    time.sort()  # Ensure timestamps are sorted
    for i in range(1, len(time)):
        time_diff = time[i] - time[i - 1]
        time_diffrence1.append(client_id)
        time_diffrence2.append(time_diff)
        #print(f"Client ID: {client_id}, Time difference: {time_diff}")

In [17]:
time_differ1 = pd.DataFrame(time_diffrence1)
time_differ2 = pd.DataFrame(time_diffrence2)

In [18]:
# Set the column names for both dataframes
time_differ2.columns = ['time_diffrence']
time_differ1.columns = ['client_id']

In [19]:
# Add them
time_differ1 = pd.concat([time_differ1, time_differ2], axis = 1)

In [21]:
# total_time spent is equal/ same  with time deffrence
time_differ1.head()

,client_id,time_diffrence
0,555,0 days 00:00:07
1,555,0 days 00:00:32
2,555,0 days 00:01:39
3,555,0 days 00:00:20
4,647,0 days 00:00:07


In [20]:
# total time spent 
total_time = time_differ1['time_diffrence'].sum()
total_time

Timedelta('1624 days 05:10:41')

In [22]:
# average time spent 
avg_time = time_differ1['time_diffrence'].mean()
avg_time

Timedelta('0 days 00:15:37.548376536')

In [23]:
# Let's merge time_diffre1 with test_merge based on client_id
test_diffre = pd.merge(test_sorted, time_differ1, on = 'client_id')

In [25]:
test_diffre.head(2)

,client_id,visitor_id,visit_id,process_step,date_time,date,time,year,month,day,...,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,age_category,variation_test,time_diffrence
0,555,402506806_56087378777,637149525_38041617439_716659,confirm,4/15/2017 13:00,"2017, 00, 15","13, 00, 34",2017,4,15,...,46,30,U,2,25454,2,6,14-30,Test,0 days 00:00:07
1,555,402506806_56087378777,637149525_38041617439_716659,confirm,4/15/2017 13:00,"2017, 00, 15","13, 00, 34",2017,4,15,...,46,30,U,2,25454,2,6,14-30,Test,0 days 00:00:32


#### Time spent on each step

In [26]:
# Convert timestamp column to datetime
test_diffre['date_time'] = pd.to_datetime(test_diffre['date_time'])

# Sort DataFrame by client ID and timestamp
test_diffre = test_diffre.sort_values(by=['client_id', 'date_time'])

# Calculate time spent on each step for each client
test_diffre['date_time'] = test_diffre.groupby('client_id')['date_time'].diff().dt.total_seconds()

# Aggregate time spent on each step for each client
time_spent_per_step = test_diffre.groupby(['client_id', 'process_step'])['date_time'].sum().reset_index()

print(time_spent_per_step)

        client_id process_step  date_time
0             555      confirm      120.0
1             555        start        0.0
2             555       step_1       60.0
3             555       step_2        0.0
4             555       step_3        0.0
...           ...          ...        ...
111339    9999729       step_1      120.0
111340    9999729       step_2      300.0
111341    9999729       step_3        0.0
111342    9999832        start        0.0
111343    9999832       step_1        0.0

[111344 rows x 3 columns]


####  Error rate

In [30]:
# display the unique client_id's
# Get unique client_ids
unique_client_ids = test_sorted['client_id'].unique()
# Iterate over each unique client_id
for client_id in unique_client_ids:
    # Extract rows based on the current client_id
    client_rows = test_sorted[test_sorted['client_id'] == client_id]

In [31]:
# To get the number of attempts a client took and number of confirmed events
def att_conf(unique_client_ids):
    for client_id in unique_client_ids:
       attempt = test_sorted[test_sorted['client_id'] == client_id] # Extract the rows for each client_Id
       len_attempt = len(test_sorted[test_sorted['client_id'] == client_id]) # Sum of how many times apper each client_id in the dataframe
       num_attempt = len_attempt/5 # How many events(True or False) are there in each 5 steps (how many full process were there?)
       num_confirm = len(attempt [attempt['process_step'] == 'confirm']) # How many confirm events are there in a specific client_id
       print(client_id, ',' , num_attempt,',', num_confirm)

In [ ]:
# display the unique client_id with the attempts they taken and confirmation  
#att_conf(unique_client_ids)

In [36]:
# Let's import our unique client_id with their attempt and confirm event 
test_unique_id = pd.read_csv('test_unique_client_id.csv')
test_unique_id.head(2)

,client_id,num_attempt,num_confirm,is_confirmation
0,555,1.0,1,True
1,647,1.0,1,True


In [39]:
# Check how many unique client_id we have in test
test_unique_id.shape

(26963, 4)

In [40]:
# Findout the confirmed process and the failed one
test_unique_id['is_confirmation'].value_counts()

is_confirmation
True     18683
False     8280
Name: count, dtype: int64

In [37]:
# Here we will extract client_id with the date_time in 5 steps of interval.
df_error = test_sorted[['client_id','date_time']]
extracted_rows = df_error.iloc[::5]
extracted_rows.head()

,client_id,date_time
0,555,4/15/2017 13:00
5,647,4/12/2017 15:47
10,934,4/18/2017 2:36
15,1336,5/8/2017 6:05
20,1336,5/8/2017 8:23


In [42]:
# Let's merge the dataset
df_err_rate = pd.merge(extracted_rows, test_unique_id, on = "client_id")
df_err_rate.head(2)

,client_id,date_time,num_attempt,num_confirm,is_confirmation
0,555,4/15/2017 13:00,1.0,1,True
1,647,4/12/2017 15:47,1.0,1,True


In [44]:
df_err_rate.shape

(35332, 5)

In [45]:
df_err_rate['is_confirmation'].value_counts()

is_confirmation
True     27067
False     8265
Name: count, dtype: int64

First let us try to find out the error and confirmation rate based on the merged data then with dataset having unique client_id.

In [49]:
# With the merged dataset
total_submissions = 35332
confirmed_submissions = 27067
error_submissions = 8265
error_rate = round((error_submissions / total_submissions) * 100)
confirmation_rate = round((confirmed_submissions / total_submissions) * 100)
print(error_rate, "||", confirmation_rate )

23 || 77


In [50]:
# With the only unique client_id dataframe
total_submissions = 26963
confirmed_submissions = 18683
error_submissions = 8280
error_rate = round((error_submissions / total_submissions) * 100)
confirmation_rate = round((confirmed_submissions / total_submissions) * 100)
print(error_rate, "||", confirmation_rate )

31 || 69


#### Hypothesis testing (verfing our kpi out put statstically)